## EXTRACCIÓN DE DATOS:


In [1]:
from selenium import webdriver
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import re

Comenzamos realizando web scraping de las normativas autonómicas en materia de violencia de género para obtener un dataframe donde tengamos las regiones autonómicas y los años en que ha existido nueva norma o modificación en este ámbito.

Encontramos la publicación del BOE donde vienen recogidas las normativas relacionadas con esta materia (Resolución de 16 de marzo de 2023, de la Secretaría de Estado de Igualdad y contra la Violencia de Género, por la que se publica el Acuerdo de la Conferencia Sectorial de Igualdad, de 3 de marzo de 2023, por el que se aprueba el plan conjunto plurianual en materia de violencia contra las mujeres (2023-2027)) y se procede al scrapeo de la misma.

In [2]:
opciones=Options()
PATH = "/driver/chromedriver"
from selenium.webdriver.common.by import By
driver = webdriver.Chrome(opciones)

In [3]:
url = 'https://www.boe.es/diario_boe/txt.php?id=BOE-A-2023-7326'

In [4]:
driver.get(url)

In [5]:
lista = driver.find_elements(By.XPATH, '//p[@class="parrafo_2" or contains(@class, "parrafo")]')

In [6]:
lista

[<selenium.webdriver.remote.webelement.WebElement (session="e8effba03926fdf7bf57861b54afb21d", element="89762E49C7E71A7311BB6EAE6D4E9FBD_element_42")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e8effba03926fdf7bf57861b54afb21d", element="89762E49C7E71A7311BB6EAE6D4E9FBD_element_43")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e8effba03926fdf7bf57861b54afb21d", element="89762E49C7E71A7311BB6EAE6D4E9FBD_element_44")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e8effba03926fdf7bf57861b54afb21d", element="89762E49C7E71A7311BB6EAE6D4E9FBD_element_45")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e8effba03926fdf7bf57861b54afb21d", element="89762E49C7E71A7311BB6EAE6D4E9FBD_element_46")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e8effba03926fdf7bf57861b54afb21d", element="89762E49C7E71A7311BB6EAE6D4E9FBD_element_47")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e8effba03926fdf7bf57861b54af

In [7]:
# Vamos a extraer únicamente los párrafos que contienen las normativas autonómicas:

lst2=[]
for i in range(231, 271):
    lst2.append(lista[i].text)

In [8]:
driver.quit()

In [9]:
lst2

['–\u2002País Vasco:',
 '●\u2002Ley 4/2005, de 18 de febrero, para la Igualdad de Mujeres y Hombres, modificada por la Ley 1/2022, de 3 de marzo, de segunda modificación de la Ley para la Igualdad de Mujeres y Hombres.',
 '–\u2002Cataluña:',
 '●\u2002Ley 5/2008, de 24 de abril, del derecho de las mujeres a erradicar la violencia machista, modificada por la Ley 17/2020, de 22 de diciembre, de modificación de la Ley 5/2008, del derecho de las mujeres a erradicar la violencia machista.',
 '●\u2002Ley 17/2015, de 21 de julio, de igualdad efectiva de mujeres y hombres',
 '–\u2002Galicia:',
 '●\u2002Ley 11/2007, de 27 de julio, gallega para la prevención y el tratamiento integral de la violencia de género, modificada por la Ley 15/2021, de 3 de diciembre, por la que se modifica la Ley 11/2007, de 27 de julio, gallega para la prevención y el tratamiento integral de la violencia de género.',
 '●\u2002Decreto Legislativo 2/2015, de 12 de febrero, por el que se aprueba el texto refundido de las 

In [10]:
lst2[0]

'–\u2002País Vasco:'

In [11]:
# Limpieza de la lista obtenida:

for i, e in enumerate(lst2):
    if e.startswith('–\u2002'):
        lst2[i] = e.split('–\u2002', 1)[1]
    elif e.startswith('●\u2002'):
        lst2[i] = e.split('●\u2002', 1)[1]


In [12]:
lst2

['País Vasco:',
 'Ley 4/2005, de 18 de febrero, para la Igualdad de Mujeres y Hombres, modificada por la Ley 1/2022, de 3 de marzo, de segunda modificación de la Ley para la Igualdad de Mujeres y Hombres.',
 'Cataluña:',
 'Ley 5/2008, de 24 de abril, del derecho de las mujeres a erradicar la violencia machista, modificada por la Ley 17/2020, de 22 de diciembre, de modificación de la Ley 5/2008, del derecho de las mujeres a erradicar la violencia machista.',
 'Ley 17/2015, de 21 de julio, de igualdad efectiva de mujeres y hombres',
 'Galicia:',
 'Ley 11/2007, de 27 de julio, gallega para la prevención y el tratamiento integral de la violencia de género, modificada por la Ley 15/2021, de 3 de diciembre, por la que se modifica la Ley 11/2007, de 27 de julio, gallega para la prevención y el tratamiento integral de la violencia de género.',
 'Decreto Legislativo 2/2015, de 12 de febrero, por el que se aprueba el texto refundido de las disposiciones legales de la Comunidad Autónoma de Galici

In [13]:
# Transformación en un diccionario:

leyes = {}
region = None  # Variable para almacenar la región actual mientras iteramos por la lista

for e in lst2:
    if e.endswith(':'):  # Si la cadena termina en ':', es el nombre de una región
        region = e.rstrip(':').strip()  # Obtenemos el nombre de la región sin los dos puntos y con espacios eliminados
        leyes[region] = []  # Inicializamos la lista de leyes para esta región
    elif region is not None:  # Si no termina en ':', y ya hemos encontrado una región
        leyes[region].append(e.strip())

In [14]:
leyes

{'País Vasco': ['Ley 4/2005, de 18 de febrero, para la Igualdad de Mujeres y Hombres, modificada por la Ley 1/2022, de 3 de marzo, de segunda modificación de la Ley para la Igualdad de Mujeres y Hombres.'],
 'Cataluña': ['Ley 5/2008, de 24 de abril, del derecho de las mujeres a erradicar la violencia machista, modificada por la Ley 17/2020, de 22 de diciembre, de modificación de la Ley 5/2008, del derecho de las mujeres a erradicar la violencia machista.',
  'Ley 17/2015, de 21 de julio, de igualdad efectiva de mujeres y hombres'],
 'Galicia': ['Ley 11/2007, de 27 de julio, gallega para la prevención y el tratamiento integral de la violencia de género, modificada por la Ley 15/2021, de 3 de diciembre, por la que se modifica la Ley 11/2007, de 27 de julio, gallega para la prevención y el tratamiento integral de la violencia de género.',
  'Decreto Legislativo 2/2015, de 12 de febrero, por el que se aprueba el texto refundido de las disposiciones legales de la Comunidad Autónoma de Galic

In [17]:
años_leyes = {}

# Expresión regular para encontrar años de cuatro dígitos
año = re.compile(r'\b\d{4}\b')

for region, leyes_lista in leyes.items():
    # Utilizamos un conjunto para evitar años duplicados
    años = set()
    
    # Extraemos los años usando expresiones regulares
    años.update({int(match.group()) for ley in leyes_lista for match in año.finditer(ley)})
    
    # Almacenamos la información en el nuevo diccionario
    años_leyes[region] = list(años)

In [18]:
años_leyes

{'País Vasco': [2005, 2022],
 'Cataluña': [2008, 2020, 2015],
 'Galicia': [2016, 2015, 2021, 2007],
 'Andalucía': [2018, 2007],
 'Principado de Asturias': [2011],
 'Cantabria': [2004],
 'La Rioja': [2022],
 'Región de Murcia': [2019, 2007],
 'Comunitat Valenciana': [2012],
 'Aragón': [2007],
 'Castilla-La Mancha': [2010, 2018],
 'Canarias': [2017, 2003],
 'Comunidad Foral de Navarra': [2018, 2019, 2015],
 'Extremadura': [2011],
 'Islas Baleares': [2016],
 'Comunidad de Madrid': [2018, 2005],
 'Castilla y León': [2017, 2010, 2003]}

In [23]:
total_años = sorted(set(año for años in años_leyes.values() for año in años))

# Crear un DataFrame inicializado con ceros
normativas = pd.DataFrame(0, index = años_leyes.keys(), columns=total_años)

# Llenar el DataFrame con unos donde corresponda
for region, años in años_leyes.items():
    normativas.loc[region, años] = 1

In [25]:
normativas

,2003,2004,2005,2007,2008,2010,2011,2012,2015,2016,2017,2018,2019,2020,2021,2022
País Vasco,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
Cataluña,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
Galicia,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0
Andalucía,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
Principado de Asturias,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
Cantabria,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
La Rioja,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Región de Murcia,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
Comunitat Valenciana,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
Aragón,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
# Guardamos el archivo para su posterior análisis:

normativas.to_csv('../data/normativas.csv', index=True)